In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import duckdb

In [17]:
!pip install -U duckdb==0.8.1

Connection with the trusted_database.db takes place and it prints the names of the tables it contains. In the next tables continue some exploration of the data. This notebook is used for the exploitation zone. We separate this zone in two notebooks. In this one the homogenization of data takes place while in the other the entities are created.

In [18]:
#pip install unidecode

In [2]:
duckdb.__version__

'0.8.1'

In [3]:
import os
import pandas as pd
import duckdb
import sys
import unidecode

#con = duckdb.connect(database='/content/drive/MyDrive/ADSDB/TrustedZone/trusted_database.db')
con = duckdb.connect(database='trusted_database.db')
result = con.execute('SHOW TABLES')
tables = result.fetchall()

table_words = []
for table in tables:
    table_words.append(table[0].split('_')[0])
tbl_words = list(set(table_words))
print(tbl_words)

['rents', 'income']


Firstly, convert sql tables to dataframes and print the first 5 rows of 'income' and 'rents' table

In [4]:
table_name = 'income'

query = f'SELECT * FROM {table_name};'
df_income = pd.read_sql_query(query, con)
print(df_income.head())

table_name = 'rents'

query = f'SELECT * FROM {table_name};'
df_rents = pd.read_sql_query(query, con)
print(df_rents.head())
con.close()

    Any  Codi_Districte Nom_Districte  Codi_Barri Nom_Barri  Seccio_Censal  \
0  2018               1  Ciutat Vella           1  el Raval              1   
1  2018               1  Ciutat Vella           1  el Raval              2   
2  2018               1  Ciutat Vella           1  el Raval              3   
3  2018               1  Ciutat Vella           1  el Raval              4   
4  2018               1  Ciutat Vella           1  el Raval              5   

   Import_Renda_Bruta_  id  
0                12391   1  
1                 9577   2  
2                 9833   3  
3                12977   4  
4                10142   5  
   Year  Trimester      District                          Neighbourhood  \
0  2018          1  Ciutat Vella                               el Raval   
1  2018          1  Ciutat Vella                         Gothic Quarter   
2  2018          1  Ciutat Vella                         la Barceloneta   
3  2018          1  Ciutat Vella  Sant Pere, Santa Cateri

c:\users\sayye\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\users\sayye\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
df_income['Nom_Districte'].value_counts()  # all the different Districts in income table along with their frequencies

Sant Martí             441
Horta-Guinardó         369
Sants-Montjuïc         351
Nou Barris             351
Eixample               346
Sarrià-Sant Gervasi    294
Sant Andreu            285
Gràcia                 261
L'Eixample             173
Les Corts              171
Ciutat Vella           162
Name: Nom_Districte, dtype: int64

In [6]:
df_rents['District'].value_counts()   # all the different Districts in rents table along with their frequencies

Nou Barris             242
Horta-Guinardo         240
Sant Marti             240
Sants-Montjuic         170
Sant Andreu            148
Eixample               144
Sarria-Sant Gervasi    144
Gracia                 120
Ciutat Vella            96
Les Corts               72
Name: District, dtype: int64

In [7]:
barrios_income = df_income['Nom_Barri'].value_counts()  # find and save the different Neighbourhoods and their frequencies of the income table

In [8]:
barrios_income.head(5)

la Nova Esquerra de l'Eixample    120
Sant Andreu                       117
les Corts                         105
la Sagrada Família                102
Sants                              87
Name: Nom_Barri, dtype: int64

In [9]:
barrios_rents = df_rents['Neighbourhood'].value_counts()    # find and save the different Neighbourhoods and their frequencies of the rents table

In [10]:
barrios_rents

el Raval                      24
Ciutat Meridiana              24
el Carmel                     24
Gothic Quarter                24
Sant Genis dels Agudells      24
                              ..
la Sagrada Familia             8
Sant Martíide Provencals       8
Torre Baro                     4
Baro de Viver                  4
la Marina del Prat Vermell     2
Name: Neighbourhood, Length: 72, dtype: int64

In this phase we are testing which are the common neighbourhoods and districts of the two tables, and if they are referring to the same neighbourhood, but differently written, convert them to the same format. The purpose of this is to homogenize the two tables so they can be integrated.

In [11]:
barrios_income[barrios_income.isin(barrios_rents)]  # only 3 neighbourhoods are common. That seems odd. Thus, we are processing the values.

Verdun          24
la Teixonera    24
Sants-Badal     16
Name: Nom_Barri, dtype: int64

In [12]:
import unidecode

Strip whitespaces before and after string, convert to lowercase and unidecode to english characters without accents and special characters. Do this for neighbourhoods and districts in both datasets

In [13]:
df_income['Nom_Barri'] = df_income['Nom_Barri'].apply(lambda x: x.strip())
df_income['Nom_Barri'] = df_income['Nom_Barri'].apply(lambda x: x.lower())
df_income['Nom_Barri'] = df_income['Nom_Barri'].apply(lambda x: unidecode.unidecode(x))

In [14]:
df_income['Nom_Barri'].value_counts()

la nova esquerra de l'eixample    120
sant andreu                       117
les corts                         105
la sagrada familia                102
sants                              87
                                 ... 
baro de viver                       6
torre baro                          6
la clota                            3
vallbona                            3
la marina del prat vermell          3
Name: Nom_Barri, Length: 80, dtype: int64

In [15]:
df_rents['Neighbourhood'].value_counts()

el Raval                      24
Ciutat Meridiana              24
el Carmel                     24
Gothic Quarter                24
Sant Genis dels Agudells      24
                              ..
la Sagrada Familia             8
Sant Martíide Provencals       8
Torre Baro                     4
Baro de Viver                  4
la Marina del Prat Vermell     2
Name: Neighbourhood, Length: 72, dtype: int64

In [16]:
df_rents['Neighbourhood'] = df_rents['Neighbourhood'].apply(lambda x: x.strip())
df_rents['Neighbourhood'] = df_rents['Neighbourhood'].apply(lambda x: x.lower())
df_rents['Neighbourhood'] = df_rents['Neighbourhood'].apply(lambda x: unidecode.unidecode(x))

In [17]:
df_rents['Neighbourhood'].value_counts()

el raval                      24
ciutat meridiana              24
el carmel                     24
gothic quarter                24
sant genis dels agudells      24
                              ..
la sagrada familia             8
sant martiide provencals       8
torre baro                     4
baro de viver                  4
la marina del prat vermell     2
Name: Neighbourhood, Length: 72, dtype: int64

Check now the common neighbourhoods in rents and income tables

In [18]:
common_barrios = list(set(df_income['Nom_Barri']) & set(df_rents['Neighbourhood']))

In [19]:
len(common_barrios)  # there are 68 common neighbourhoods

68

Then check, which neighbourhoods are only in the income table and then only in the rents table. We observer that both have sagrada familia, gotic and fort pienc, but they are written differently. Convert them to same format. In addition, income table has sagrada familia and gotic written in two different formats. Convert them to one.

In [20]:
df_income['Nom_Barri'][~df_income['Nom_Barri'].isin(common_barrios)].value_counts()

vila de gracia               72
el fort pienc                60
camp de l'arpa del clot      50
el putget i farro            38
sant gervasi- galvany        31
el gotic                     18
sant gervasi- la bonanova    18
sants-badal                  16
el barri gotic                9
can peguera                   6
la clota                      3
vallbona                      3
Name: Nom_Barri, dtype: int64

In [21]:
df_rents['Neighbourhood'][~df_rents['Neighbourhood'].isin(common_barrios)].value_counts()

gothic quarter              24
fort pienc                  24
sagrada familia             16
sant martiide provencals     8
Name: Neighbourhood, dtype: int64

In [22]:
df_rents['Neighbourhood'] = df_rents['Neighbourhood'].str.replace('la sagrada familia', 'sagrada familia')
df_rents['Neighbourhood'] = df_rents['Neighbourhood'].str.replace('gothic quarter', 'el gotic')
df_rents['Neighbourhood'] = df_rents['Neighbourhood'].str.replace('fort pienc', 'el fort pienc')

df_income['Nom_Barri'] = df_income['Nom_Barri'].str.replace('el barri gotic', 'el gotic')
df_income['Nom_Barri'] = df_income['Nom_Barri'].str.replace('la sagrada familia', 'sagrada familia')

After these changes, there are 70 common neighbourhoods, income table has 9 extra and rents table one extra.

In [23]:
common_barrios = list(set(df_income['Nom_Barri']) & set(df_rents['Neighbourhood']))

In [24]:
len(common_barrios)

70

Now, similar process for finding common and different districts in the two tables. They have the same 10 districts, but eixample is written differently. Convert to same string format.

In [25]:
df_income['Nom_Districte'] = df_income['Nom_Districte'].apply(lambda x: x.strip())
df_income['Nom_Districte'] = df_income['Nom_Districte'].apply(lambda x: x.lower())
df_income['Nom_Districte'] = df_income['Nom_Districte'].apply(lambda x: unidecode.unidecode(x))
df_income['Nom_Districte'] = df_income['Nom_Districte'].str.replace("l'eixample", 'eixample')
df_income['Nom_Districte'].value_counts()

eixample               519
sant marti             441
horta-guinardo         369
sants-montjuic         351
nou barris             351
sarria-sant gervasi    294
sant andreu            285
gracia                 261
les corts              171
ciutat vella           162
Name: Nom_Districte, dtype: int64

In [26]:
df_rents['District'] = df_rents['District'].apply(lambda x: x.strip())
df_rents['District'] = df_rents['District'].apply(lambda x: x.lower())
df_rents['District'] = df_rents['District'].apply(lambda x: unidecode.unidecode(x))
#df_rents['District'] = df_rents['District'].str.replace('eixample', "l'eixample")
df_rents['District'].value_counts()

nou barris             242
horta-guinardo         240
sant marti             240
sants-montjuic         170
sant andreu            148
eixample               144
sarria-sant gervasi    144
gracia                 120
ciutat vella            96
les corts               72
Name: District, dtype: int64

In [27]:
common_districts = list(set(df_rents['District']) & set(df_income['Nom_Districte']))
len(common_districts)

10

In [28]:
df_rents['District'][~df_rents['District'].isin(common_districts)].value_counts()

Series([], Name: District, dtype: int64)

In [29]:
df_income['Nom_Districte'][~df_income['Nom_Districte'].isin(common_districts)].value_counts()

Series([], Name: Nom_Districte, dtype: int64)

In [30]:
df_income.head()

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Renda_Bruta_,id
0,2018,1,ciutat vella,1,el raval,1,12391,1
1,2018,1,ciutat vella,1,el raval,2,9577,2
2,2018,1,ciutat vella,1,el raval,3,9833,3
3,2018,1,ciutat vella,1,el raval,4,12977,4
4,2018,1,ciutat vella,1,el raval,5,10142,5


In [31]:
average_income_by_barri = df_income.groupby(['Any','Nom_Districte', 'Nom_Barri'])['Import_Renda_Bruta_'].mean().reset_index()

# Display the result
print(average_income_by_barri)

      Any        Nom_Districte                              Nom_Barri  \
0    2018         ciutat vella                               el gotic   
1    2018         ciutat vella                               el raval   
2    2018         ciutat vella                         la barceloneta   
3    2018         ciutat vella  sant pere, santa caterina i la ribera   
4    2018             eixample                          el fort pienc   
..    ...                  ...                                    ...   
214  2020  sarria-sant gervasi                        les tres torres   
215  2020  sarria-sant gervasi                  sant gervasi- galvany   
216  2020  sarria-sant gervasi              sant gervasi- la bonanova   
217  2020  sarria-sant gervasi                                 sarria   
218  2020  sarria-sant gervasi  vallvidrera, el tibidabo i les planes   

     Import_Renda_Bruta_  
0           15882.444444  
1           11238.095238  
2           14431.090909  
3           168

In [32]:
average_income_by_barri.head()

,Any,Nom_Districte,Nom_Barri,Import_Renda_Bruta_
0,2018,ciutat vella,el gotic,15882.444444
1,2018,ciutat vella,el raval,11238.095238
2,2018,ciutat vella,la barceloneta,14431.090909
3,2018,ciutat vella,"sant pere, santa caterina i la ribera",16840.307692
4,2018,eixample,el fort pienc,21270.550000


In [33]:
df_rents.shape

(1616, 7)

In [34]:
df_rents['Average__rent'].value_counts()

average rent (euro/month)             808
average rent per surface (euro/m2)    808
Name: Average__rent, dtype: int64

In [35]:
df_rents = df_rents.loc[df_rents['Average__rent'] == 'average rent (euro/month)']
df_rents.reset_index(drop=True)
df_rents = df_rents[['Year','Trimester','District','Neighbourhood','Price']]
print(df_rents.shape)
df_rents.head()

(808, 5)


,Year,Trimester,District,Neighbourhood,Price
0,2018,1,ciutat vella,el raval,792.74
1,2018,1,ciutat vella,el gotic,998.40
2,2018,1,ciutat vella,la barceloneta,870.84
3,2018,1,ciutat vella,"sant pere, santa caterina i la ribera",923.44
4,2018,1,eixample,el fort pienc,910.53


In [36]:
df_rents.to_csv('df_rent_avg_month.csv',index=False) 
average_income_by_barri.to_csv('df_income_avg.csv',index=False)